# Our modeling approach

- We fully characterize the existence of a 1-round protocol by a Mixed Integer Linear Program.


In [1]:
from sage.all import *
import itertools
import random
from sage.sat.solvers.satsolver import SAT
from sage.sat.solvers.cryptominisat import CryptoMiniSat
from sage.misc.temporary_file import atomic_write
import copy
import time

In [2]:
solver = SAT(solver="LP")
solver.add_clause((-1,2))
solver.add_clause((1,3))
solution = solver()
print ' solution =',solution
lst = ['play', 'ground']
print 'ground' in lst

m = matrix(QQ, [[1,2],[4,2],[11,3]])
print 'm = ',m
x = []
for v in m:
    x.append(v)
    
x = matrix(QQ, x)
print 'x = ',x
 
pr = Permutations(20).random_element()
print pr, pr[4], pr[7]


v = vector(QQ, [2,3,4])
print 'len = ',len(v)

min(5, 44)

l = [1,2,3,4,5]
print l [2:4]
(5 == 5) + 0


 solution = [None, False, False, True]
True
m =  [ 1  2]
[ 4  2]
[11  3]
x =  [ 1  2]
[ 4  2]
[11  3]
[16, 12, 5, 13, 8, 11, 3, 1, 10, 7, 4, 18, 6, 2, 15, 20, 14, 19, 17, 9] 8 1
len =  3
[3, 4]


1

In [3]:
# TODO: check correctness etc
def get_at(tup, a, l):
    """Given 2*l elements flattened into a tuple, pick 1 out of each pair according to array of bits."""
    assert len(tup) == 2*l
    assert len(a) == l
    for bit in a:
        assert bit in (0, 1)

    # tup structure is: $(t_{0,0}, t_{0,1}, t_{1,0}, t_{1,1}, t_{2,0}, t_{2,1})$
    return tuple(tup[2*i + a[i]] for i in range(l))

assert get_at((3, 13, 85, 95), (0, 1), 2) == (3, 95)

In [34]:
 
def tuples_fixed_proj(l, proj_ind, proj_val):
    tuples = list()
    x = [0 for i in range(2*l)]
    for i in range(l):
        x[i*2 + proj_ind[i]] = proj_val[i]
    for g in itertools.product(range(2), repeat = l):
        for i in range(l):
            x[i*2 + 1 - proj_ind[i]] = g[i]
        tuples.append(tuple(x[:]))
    return tuples     

                            
def get_rand_tup(len):
    return tuple([sage.misc.prandom.choice(range(2)) for i in range(len)])
                

# i = -1 , search for an index
# i = -2 : auxiliary for relaxed mode
# [1,0,0]
# [0,1,0]
# [0,0,1]
# [0,0,0]

def safe_insert(val_l, ind_l, (i,ind), v):
  #  print 'safe-inserting ',val_l, ind_l, (i,ind), v
    to_search = []
    if i >= 0:
        to_search = [i]
    elif i == -1:
        to_search = [j for j in range(3)]
    elif i == -2:
        val_l[ind_l.index(ind)] = v
        return
    
    for j in to_search:
        if (j,ind) in ind_l:
            val_l[ind_l.index((j,ind))] = v
      #      print 'inserted !',(i,ind)



def permute_rows(m, b):
    
    nrows = m.nrows()
    perm = Permutations(nrows).random_element()
    pm = [row for row in m]
    pb = [v for v in b]
    
    new_m = matrix(QQ, [pm[perm[i] - 1] for i in range(nrows)])
    
    new_b = vector(QQ, [pb[perm[i] - 1] for i in range(nrows)])
    
    return (new_m, new_b)
    
    
def get_full_rank_sub(m, b):
   
    nrows = m.nrows()
    new_mat = []
    new_b = []
    
 #   print 'into get_full_rank_sub',m.nrows(), m.ncols()
    
    for (i,row) in enumerate(m):
        so_far = (matrix(QQ, new_mat)).transpose()
        try:
            so_far\row
        except Exception as e:
#            print 'at full rank sub Exception = ',e,i
            new_mat.append(row)
            new_b.append(b[i])
        
    new_mat = matrix(QQ, new_mat)
    new_b = vector(new_b)
    
    return (new_mat, new_b)


def print_matrix(m):

    if m.ncols < 37:
        print m
        return
    
    per_row = 30
    for row in m:
        n = ceil(QQ(m.nrows())/per_row)
        for i in range(n):
            ran = min(per_row, m.nrows() - i * per_row)
            print [row[i * per_row + j] for j in range(ran)]
        print '\n'    
          

def non_zeros(m):            
    return sum(sum(1 for j in row if abs(j) != 0) for row in m)        
        

# only captures a small subset of constraints                
def test_linear_solutions(l, verbose = 0):
    
        b = []
        constraints = []
        
        if verbose:
            print 'generating constraints ...'
        
        
        a_set = []
   #     a_only_set = [[],[],[]]
        for a in itertools.product(range(2), repeat = l):
            for g in itertools.product(range(2), repeat = l):
                for z in range(2):
                    a_set.append((a,g,z))

                       
        a_set_len = len(a_set) 
        if verbose:
            print '|List0|=', a_set_len
        
        # well-formed distribution - relaxed
        
        cur_row = [0 for j in range(a_set_len)]
        
        for (a, g, z) in a_set:
            if g == zeros:
                cur_row[a_set.index((a, g, z))] = 1   

        constraints.append(cur_row)
        b.append(3)

        # require equiality of distributions
        for a in itertools.product(range(2), repeat = l):
            for g in itertools.product(range(2), repeat = l): 
                if g != zeros:
                    cur_row = [0 for j in range(a_set_len)]
                    # We are guaranteed that a is not completely out of the 
                    # set. If it was, eccidentally, this will make the set unsolvable
                    cur_row[a_set.index((a, zeros, 0))] = 1
                    cur_row[a_set.index((a, zeros, 1))] = 1
                    cur_row[a_set.index((a, g, 0))] = -1
                    cur_row[a_set.index((a, g, 1))] = -1
                    
                    b.append(0)
                    constraints.append(cur_row)

        
        # require security against a malicious server
        
        extra_constraints = []
        extra_b = []
        
        for t in itertools.product(range(2), repeat = 2*l):
            cur_row = [0 for i in range(a_set_len)]
            for a in itertools.product(range(2), repeat = l):
                cur_row[a_set.index((a, get_at(t,a,l), 1))] = 1
                
            extra_constraints.append(cur_row)
            extra_b.append(1)
                
        
        m_constr = matrix(QQ,constraints + extra_constraints)
        right_side = vector(QQ,b + extra_b)
        
        solver = MixedIntegerLinearProgram(maximization=True, solver='ppl')
        prot = solver.new_variable(nonnegative=True)
        aux = solver.new_variable(nonnegative=True)
        
        
        t_list = [a for a in itertools.product(range(2), repeat = 2*l)]
        
        if verbose:
            print 'Created linear system of rank',m_constr.rank(),'/',m_constr.nrows(), m_constr.ncols(),' vs ',a_set_len
      #      print m_constr
        try:
            v = m_constr\right_side
#            ker = m_constr.right_kernel()        
            print 'solution for linear system = ',v

            print 'len(solution) = ',len(v)
            nz = sum(1 for x in v if abs(x) > 0)
            
            print 'non-zeros ', nz
            
            if verbose:
                print '==================================================V'
                print 'Now looking for a well-formed solution, not just any'
            
            for (i,cur_row) in enumerate(constraints):
                solver.add_constraint(sum(cur_row[j]*prot[j] for j in range(a_set_len)) == b[i])
             
            for (i,cur_row) in enumerate(extra_constraints):
           #     print 'cur_row = ', cur_row
                solver.add_constraint(aux[i] >= 1 - sum(cur_row[j]*prot[j] for j in range(a_set_len)))
                solver.add_constraint(aux[i] >= -1 + sum(cur_row[j]*prot[j] for j in range(a_set_len)))
             
            
            solver.set_objective(-sum(aux[i] for i in range(len(extra_b))))
            
            res = solver.solve()  
            
            print 'The optimal error sum = ',-res
            
            print 'Prot variables = '
            print [(a_set[i],solver.get_values(prot[i])) for i in range(a_set_len) if solver.get_values(prot[i]) > 0]
            
            print 'Aux variables = '
            aux_l = [(t_list[i], solver.get_values(aux[i])) for i in range(pow(2, 2*l)) if solver.get_values(aux[i]) > 0]
            print aux_l
            print 'len(aux_l) = ',len(aux_l)
            
            
            print '==================================================V'
        except Exception as e:
            print 'Exception = ',e
            if verbose:
                print '==================================================X'
            return -1
        
 #       print 'Returning ',(nz, a_set_len)
        return res        


# global variables

my_eps = QQ(1/10000)

In [35]:
# main     
import time 


def main(dim):

    t0 = time.time()

    try:
        test_linear_solutions(l, 1)

    finally:
        print 'time =', time.time() - t0,'\n'



In [ ]:
for l in range(2,5):
    print 'l = ',l
    print '!!!!!!!!!!!!!!!!!!!!!!!!!!'
    zeros = tuple((0 for i in range(l)))
    main (l)

l =  2
!!!!!!!!!!!!!!!!!!!!!!!!!!
generating constraints ...
|List0|= 32
Created linear system of rank 22 / 29 32  vs  32
solution for linear system =  (2, 1, 2, 1, 2, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
len(solution) =  32
non-zeros  8
==================================================V
Now looking for a well-formed solution, not just any
The optimal error sum =  0
Prot variables = 
[(((0, 0), (0, 0), 0), 1), (((0, 0), (0, 1), 1), 1), (((0, 0), (1, 0), 0), 1), (((0, 0), (1, 1), 1), 1), (((1, 0), (0, 0), 0), 1), (((1, 0), (0, 0), 1), 1), (((1, 0), (0, 1), 0), 2), (((1, 0), (1, 0), 0), 1), (((1, 0), (1, 0), 1), 1), (((1, 0), (1, 1), 0), 2)]
Aux variables = 
[]
len(aux_l) =  0
==================================================V
time = 0.07985496521 

l =  3
!!!!!!!!!!!!!!!!!!!!!!!!!!
generating constraints ...
|List0|= 128
Created linear system of rank 84 / 121 128  vs  128
solution for linear system =  (2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 